## Upload Dataset

### Import Libraries

In [1]:
import os
import re
from pathlib import Path

import pandas as pd
from tqdm import tqdm

### Set Class -> ResearchThemeParser

In [2]:
class ResearchThemeParser:

    def __init__(self):
        self.doc2label_ = None

    @staticmethod
    def get_research_theme_files():
        """
        a list with all the csv files included in the 
        research_theme/documents directory
        :return: 
        """
        return list(Path('research_theme').joinpath(
            'documents').glob('**/*.csv'))

    @property
    def doc2label(self) -> dict:
        """
        Loads lazily the doc 2 abstract research_theme assignment
        :return:
        """
        if self.doc2label_ is None:
            path = Path('research_theme').joinpath('research_theme.csv')
            data = pd.read_csv(path)
            
            # removes the .txt extension for each entry on the research_theme.csv file
            data['document'] = data['document'].apply(
                lambda x: re.sub('.txt', '', x))

            # gets the label of each row of the document
            self.doc2label_ = data.set_index('document').to_dict().get('label')

        return self.doc2label_

    def create_paper_level_dataset(self) -> pd.DataFrame:
        """
        a Dataframe of document dictionaries containing label, binary label and 
        text information 
        :return:
        """
        docs = list()
        # read the csv that is contained in the list from 
        # get_research_theme_files() list

        for fpath in tqdm(self.get_research_theme_files()):
            df = pd.read_csv(fpath)
            
            # get the finelame (last column) of each line 
            filename = str(fpath).split(os.sep)[-1]
            
            # remove csv extension
            filename = re.sub('.csv', '', filename)
            paper_corpora = ' '.join(df['sentence'])

            label = self.doc2label.get(filename)
            
            # check label 
            binary_label = label if label == 'Study' else 'Not Study'
            
            # construct a new document dictionary with the captured information
            doc = {'label': self.doc2label.get(filename),
                   'binary_label': binary_label,
                   'text': paper_corpora}
            docs.append(doc)

        return pd.DataFrame(docs)

### Run the Parser to extract data

In [3]:
if __name__ == "__main__":
    parser = ResearchThemeParser()
    paper_level_rt = parser.create_paper_level_dataset()
    print(paper_level_rt)
    print(paper_level_rt['label'].value_counts())
    print(paper_level_rt['binary_label'].value_counts())

100%|███████████████████████████████████████████████████████████████████████████████| 828/828 [00:05<00:00, 161.88it/s]

      label binary_label                                               text
0     Study        Study  Multivariate Granger causality between CO2 emi...
1     Study        Study  Electricity consumption-GDP nexus in Pakistan:...
2     Study        Study  Ecological total-factor energy efficiency of r...
3    Method    Not Study  Exploration and corrective measures of greenho...
4     Study        Study  Solutions for improving the energy efficiency ...
..      ...          ...                                                ...
823   Study        Study  How to quantify the resolution of surface clim...
824   Study        Study  A visual method to identify significant latitu...
825   Study        Study  Effect of production system and farming strate...
826   Study        Study  The effect of future climate scenarios on the ...
827   Study        Study  Comparing environmental impacts of beef produc...

[828 rows x 3 columns]
Study              630
Method              75
Review            

## Exploratory Analysis

### Overview of the data

In [4]:
# first, set the parameters of the notebook
pd.set_option('display.max_columns', 10) # display up to 10 columns
pd.set_option('display.max_rows', 10) # display up to 10 rows

display(paper_level_rt)

label binary_label                                               text
0     Study        Study  Multivariate Granger causality between CO2 emi...
1     Study        Study  Electricity consumption-GDP nexus in Pakistan:...
2     Study        Study  Ecological total-factor energy efficiency of r...
3    Method    Not Study  Exploration and corrective measures of greenho...
4     Study        Study  Solutions for improving the energy efficiency ...
..      ...          ...                                                ...
823   Study        Study  How to quantify the resolution of surface clim...
824   Study        Study  A visual method to identify significant latitu...
825   Study        Study  Effect of production system and farming strate...
826   Study        Study  The effect of future climate scenarios on the ...
827   Study        Study  Comparing environmental impacts of beef produc...

[828 rows x 3 columns]

In [5]:
# See the text of the first paper
idx = 0

print(paper_level_rt.loc[idx]['text'])

Multivariate Granger causality between CO2 emissions, energy consumption, FDI (foreign direct investment) and GDP (gross domestic product): Evidence from a panel of BRIC (Brazil, Russian Federation, India, and China) countries Abstract: This paper addresses the impact of both economic growth and financial development on environmental degradation using a panel cointegration technique for the period between 1980 and 2007, except for Russia (1992–2007). In long-run equilibrium, CO2 emissions appear to be energy consumption elastic and FDI inelastic, and the results seem to support the Environmental Kuznets Curve (EKC) hypothesis. The causality results indicate that there exists strong bidirectional causality between emissions and FDI and unidirectional strong causality running from output to FDI. The evidence seems to support the pollution haven and both the halo and scale effects. Therefore, in attracting FDI, developing countries should strictly examine the qualifications for foreign in

### Variables' types

In [6]:
paper_level_rt.dtypes

label           object
binary_label    object
text            object
dtype: object

### Dataset's size & shape

In [7]:
print("Dataset size:", len(paper_level_rt))
print('Dataset shape: {}'.format(paper_level_rt.shape))

Dataset size: 828
Dataset shape: (828, 3)


### Check for missing values

In [8]:
paper_level_rt.isna().sum() 

label           0
binary_label    0
text            0
dtype: int64

In [9]:
# Some basic description 
paper_level_rt.describe().T 

count unique                                                top  \
label          828     16                                              Study   
binary_label   828      2                                              Study   
text           828    825  Title: The Portuguese National Strategy for In...   

             freq  
label         630  
binary_label  630  
text            2

### Drop duplicate papers

In [10]:
newdata = paper_level_rt.drop_duplicates(subset=['text'], keep='last')

In [11]:
# Sanity check
newdata.describe().T 

count unique                                                top  \
label          825     16                                              Study   
binary_label   825      2                                              Study   
text           825    825  Gender norms and social norms: differences, si...   

             freq  
label         628  
binary_label  628  
text            1

Counting the frequency of the 2 unique values - **for binary classification purposes**

In [12]:
from collections import Counter

theme_counts = Counter(newdata['binary_label'])
theme_counts.most_common()

[('Study', 628), ('Not Study', 197)]

Counting the frequency of each theme - **for multiclass classification purposes**

In [13]:
theme_counts = Counter(newdata['label'])
theme_counts.most_common()

[('Study', 628),
 ('Method', 74),
 ('Review', 29),
 ('Interview', 25),
 ('Policy', 19),
 ('Tool', 11),
 ('Prototype', 9),
 ('Treatment', 9),
 ('Evaluation', 6),
 ('Diagnostic Tool', 4),
 ('Other', 3),
 ('Clinical Trial', 3),
 ('Diagnositc Tool', 2),
 ('Search', 1),
 ('Device', 1),
 ('Infrastructure', 1)]

## Extract data

Create 2 separate dataframes for binary and multi-class classification

In [14]:
binary_clf = newdata[['binary_label', 'text']].copy()
mlb_clf = newdata[['label', 'text']].copy()

### Create a new folder in directory (provided in github)

In [15]:
# you can either download the "Dataset" directory from github, or uncomment below line and execute
# !mkdir Dataset

### Extract the dataframes to CSV files

In [16]:
path = "./Dataset/"

filename_write = os.path.join(path, "binary_clf.csv")
binary_clf.to_csv(filename_write, index=False)
print("Extraction of 'binary_clf.csv' is finished")
print('-'*50)

filename_write = os.path.join(path, "mlb_clf.csv")
mlb_clf.to_csv(filename_write, index=False)
print("Extraction of 'mlb_clf.csv' is finished")

Extraction of 'binary_clf.csv' is finished
--------------------------------------------------
Extraction of 'mlb_clf.csv' is finished
